Prerequisites for Problem

In [2]:
pip install Pulp

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pulp import *
import pulp as p

Define Input Parameters

In [5]:
# Create a list of investments
investments = ['Bonds', 'Mortgages', 'Car loans', 'Personal loans']

#Create a dictionary to save the rate of return for the investments
rate_return = {'Bonds': 0.10,
               'Mortgages':0.085,
               'Car loans':0.095,
               'Personal loans': 0.125}

# Define a parameter for the available amount of money
total_amount= 650000

Define Decision Variables

In [6]:
amount_vars= p.LpVariable.dicts("X", investments, lowBound= 0, upBound= None, cat=p.LpContinuous)
amount_vars

{'Bonds': X_Bonds,
 'Mortgages': X_Mortgages,
 'Car loans': X_Car_loans,
 'Personal loans': X_Personal_loans}

Create a variable that will contain the problem data

In [8]:
investmentProblem = p.LpProblem("Investment_Problem", p.LpMaximize)
investmentProblem

Investment_Problem:
MAXIMIZE
None
VARIABLES

Define the objective function

In [10]:
investmentProblem += p.lpSum(rate_return[i]*amount_vars[i] for i in investments)
investmentProblem

c:\Users\alipo\OneDrive\Desktop\Jupyter in VS Code\.venv\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Investment_Problem:
MAXIMIZE
0.1*X_Bonds + 0.095*X_Car_loans + 0.085*X_Mortgages + 0.125*X_Personal_loans + 0.0
VARIABLES
X_Bonds Continuous
X_Car_loans Continuous
X_Mortgages Continuous
X_Personal_loans Continuous

Define the Constraints

In [11]:
# 1. Total amount constraint: Total investment <= 650,000
investmentProblem += p.lpSum(amount_vars[i] for i in investments) <= total_amount, "Total_Investment"

# 2. Personal loan constraint: Personal loans <= 162,500
investmentProblem += amount_vars['Personal loans'] <= 0.25 * total_amount, "Personal_Loan_Limit"

# 3. Mortgages must be greater than personal loans
investmentProblem += amount_vars['Mortgages'] >= amount_vars['Personal loans'], "Mortgage_Constraint"

# 4. Bonds must be greater than personal loans
investmentProblem += amount_vars['Bonds'] >= amount_vars['Personal loans'], "Bond_Constraint"

In [12]:
investmentProblem

Investment_Problem:
MAXIMIZE
0.1*X_Bonds + 0.095*X_Car_loans + 0.085*X_Mortgages + 0.125*X_Personal_loans + 0.0
SUBJECT TO
Total_Investment: X_Bonds + X_Car_loans + X_Mortgages + X_Personal_loans
 <= 650000

Personal_Loan_Limit: X_Personal_loans <= 162500

Mortgage_Constraint: X_Mortgages - X_Personal_loans >= 0

Bond_Constraint: X_Bonds - X_Personal_loans >= 0

VARIABLES
X_Bonds Continuous
X_Car_loans Continuous
X_Mortgages Continuous
X_Personal_loans Continuous

Solve and print the status

In [13]:
status = investmentProblem.solve()
print("Status:", p.LpStatus[status])

Status: Optimal


Print the value of the objective function

In [14]:
print("Total Return = $", p.value(investmentProblem.objective))

Total Return = $ 66625.0


Print the values of the decision variables

In [15]:
for variable in amount_vars:
    print(f"{variable} = $", amount_vars[variable].varValue)

Bonds = $ 325000.0
Mortgages = $ 162500.0
Car loans = $ 0.0
Personal loans = $ 162500.0
